In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import sys

sys.path.append("../../analysisTools/")
import plotTools as ptools

## Import coffea files -- signal and background

In [2]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [3]:
# Signal
f_sig = "signal_v3"
sig_histo = util.load(f'{prefix}/{f_sig}{suffix}_mA_3m1.coffea')[0]

In [4]:
# sig_histo is a dictionary of variables
print(sig_histo.keys())

dict_keys(['sel_vtx_sign', 'cutDesc', 'cutflow', 'cutflow_cts', 'cutflow_nevts', 'cutflow_vtx_matched', 'cutflow_genEEreconstructed', 'cutflow_cts_genEEreconstructed', 'cutflow_vtx_matched_genEEreconstructed'])


## Signal

### Access the cutflow information

In [5]:
sig_samples = list(sig_histo['cutflow'].keys()) # names of signal sub-process, i.e. mass point, lifetime etc.

In [6]:
cuts = list(sig_histo['cutflow'][sig_samples[0]].keys())
print(cuts)

['all', 'hasVtx', 'cut5', 'cut6', 'cut7', 'cut8', 'cut9', 'cut10', 'cut11', 'cut12', 'cut13', 'cut14', 'cut15']


In [7]:
si = pd.DataFrame.from_dict({s:ptools.signalPoint(s) for s in sig_samples}, orient='index')

In [8]:
# cutflow
cutNames = {cname:ptools.getCut(sig_histo['cutDesc'][cname]) for cname in sig_histo['cutDesc'].keys()}

In [9]:
# signal efficiencies by delta/ctau
cf_s = sig_histo['cutflow']              # efficiency
ct_s = sig_histo['cutflow_cts']          # xsec-weighted event count
fr_s = sig_histo['cutflow_vtx_matched'] # fraction that selected vtx is truth-matched, for signal
desc = sig_histo['cutDesc']

In [10]:
# signal efficiencies by delta/ctau
raw_s = sig_histo['cutflow_nevts']

In [11]:
for process in raw_s.keys():
    print(f'{process} after preselections (raw count) = {raw_s[process]["all"]}')

sig_2018_Mchi-99p0_dMchi-18p0_ctau-100 after preselections (raw count) = 23255
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100 after preselections (raw count) = 25608
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10 after preselections (raw count) = 31933
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1 after preselections (raw count) = 31711
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100 after preselections (raw count) = 10603
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10 after preselections (raw count) = 7516
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10 after preselections (raw count) = 32183
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1 after preselections (raw count) = 48974
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100 after preselections (raw count) = 8905
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10 after preselections (raw count) = 9944
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1 after preselections (raw count) = 12770
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100 after preselections (raw count) = 7894
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10 after preselections (raw count) = 

### Access the cutflow of the signal based on mass point, lifetime, etc

In [12]:
deltas = sorted(list(set(si.delta)))

### Display the cutflow of efficiency and number of events as pandas dataframe

In [17]:
columns = list(cf_s.keys())


In [18]:
columns_ctau = {'1': {'0p1': [], '0p2': []}, '10': {'0p1': [], '0p2': []}, '100': {'0p1': [], '0p2': []}}

for name in columns:
    mchi = int(name.split('-')[1].split('p')[0])
    dmchi = int(name.split('-')[2].split('p')[0])

    delta = round(dmchi / mchi, 1)

    if 'ctau-10' not in name:
        if delta == 0.1:
            columns_ctau['1']['0p1'].append(name)
        elif delta == 0.2:
            columns_ctau['1']['0p2'].append(name)
    elif 'ctau-100' not in name:
        if delta == 0.1:
            columns_ctau['10']['0p1'].append(name)
        elif delta == 0.2:
            columns_ctau['10']['0p2'].append(name)
    else:
        if delta == 0.1:
            columns_ctau['100']['0p1'].append(name)
        elif delta == 0.2:
            columns_ctau['100']['0p2'].append(name)

ct = '10'
delta = '0p1'
print(columns_ctau[ct][delta])

['sig_2018_Mchi-94p5_dMchi-9p0_ctau-10', 'sig_2018_Mchi-84p0_dMchi-8p0_ctau-10', 'sig_2018_Mchi-73p5_dMchi-7p0_ctau-10', 'sig_2018_Mchi-63p0_dMchi-6p0_ctau-10', 'sig_2018_Mchi-52p5_dMchi-5p0_ctau-10', 'sig_2018_Mchi-42p0_dMchi-4p0_ctau-10', 'sig_2018_Mchi-31p5_dMchi-3p0_ctau-10', 'sig_2018_Mchi-21p0_dMchi-2p0_ctau-10', 'sig_2018_Mchi-10p5_dMchi-1p0_ctau-10', 'sig_2018_Mchi-105p0_dMchi-10p0_ctau-10']


In [19]:
counts = {}
purities = {}
efficiencies = {}

for ct in columns_ctau.keys():
    counts[ct] = {}
    purities[ct] = {}
    efficiencies[ct] = {}
    for delta in ['0p1', '0p2']:
        counts[ct][delta] = {}
        purities[ct][delta] = {}
        efficiencies[ct][delta] = {}
        for samp in columns_ctau[ct][delta]:
            counts[ct][delta][samp] = {}
            purities[ct][delta][samp] = {}
            efficiencies[ct][delta][samp] = {}
            for c in cuts:
                counts[ct][delta][samp][c] = ct_s[samp][c]
                purities[ct][delta][samp][c] = fr_s[samp][c]
                efficiencies[ct][delta][samp][c] = cf_s[samp][c]

In [20]:
#cts_df = {ct: pd.DataFrame.from_dict(counts[ct]).transpose() for ct in columns_ctau.keys()}

cts_df = {}
purities_df = {}
eff_df = {}

for ct in columns_ctau.keys():
    cts_df[ct] = {delta: pd.DataFrame.from_dict(counts[ct][delta]) for delta in ['0p1', '0p2']}
    purities_df[ct] = {delta: pd.DataFrame.from_dict(purities[ct][delta]) for delta in ['0p1', '0p2']}
    eff_df[ct] = {delta: pd.DataFrame.from_dict(efficiencies[ct][delta]) for delta in ['0p1', '0p2']}
    #cts_df[ct] = {delta: pd.DataFrame.from_dict(counts[ct]).transpose()}
    
    for delta in ['0p1', '0p2']:
        cts_df[ct][delta].columns = [ s.replace('sig_2018_', '') for s in columns_ctau[ct][delta] ]
        purities_df[ct][delta].columns = [ s.replace('sig_2018_', '') for s in columns_ctau[ct][delta] ]
        eff_df[ct][delta].columns = [ s.replace('sig_2018_', '') for s in columns_ctau[ct][delta] ]
    
        labels = [cutNames[c] for c in cuts]
        # Replace "No cuts" with "Preselections" and "Baseline Selection" with "0 < n(jet) < 3 & n(good vertex) > 0"
        cut = list(map(lambda x: x.replace('Baseline Selection', '0 < n(jet) < 3 & n(good vertex) > 0'), labels))
        cut = list(map(lambda x: x.replace('No cuts', 'Preselections'), cut))
        
        cts_df[ct][delta].index = cut
        purities_df[ct][delta].index = cut
        eff_df[ct][delta].index = cut

### Efficiency

In [21]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '1'
delta = '0p1'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-1,0.066110,0.017655,0.015089,0.014585,0.014582,0.014582,0.014428,0.010254,0.006304,0.004818,0.004760,0.003658,0.001974
Mchi-84p0_dMchi-8p0_ctau-1,0.058112,0.015163,0.013192,0.012714,0.012714,0.012714,0.012605,0.008974,0.005392,0.004296,0.004273,0.003326,0.001779
Mchi-73p5_dMchi-7p0_ctau-1,0.051428,0.012919,0.010991,0.010658,0.010658,0.010658,0.010549,0.007248,0.004450,0.003705,0.003668,0.002957,0.001520
Mchi-63p0_dMchi-6p0_ctau-1,0.043559,0.010812,0.009217,0.008836,0.008836,0.008836,0.008727,0.006007,0.003545,0.003002,0.002960,0.002361,0.001282
Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.008843,0.007654,0.007341,0.007341,0.007341,0.007299,0.004979,0.002818,0.002544,0.002512,0.002144,0.001238
Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.007161,0.006138,0.005858,0.005852,0.005852,0.005804,0.003931,0.002234,0.002080,0.002062,0.001806,0.001038
Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.005053,0.004338,0.004171,0.004171,0.004171,0.004137,0.002784,0.001464,0.001376,0.001371,0.001200,0.000697
Mchi-21p0_dMchi-2p0_ctau-1,0.018942,0.004017,0.003370,0.003222,0.003220,0.003220,0.003187,0.002174,0.001141,0.001086,0.001081,0.000989,0.000588
Mchi-10p5_dMchi-1p0_ctau-1,0.015928,0.003449,0.002946,0.002757,0.002755,0.002755,0.002729,0.001806,0.000903,0.000849,0.000841,0.000757,0.000416
Mchi-105p0_dMchi-10p0_ctau-1,0.073740,0.020077,0.017096,0.016562,0.016557,0.016557,0.016403,0.011799,0.007365,0.005394,0.005353,0.003924,0.002075


In [22]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '10'
delta = '0p1'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-10,0.058330,0.013406,0.011330,0.010929,0.010929,0.010929,0.010751,0.007259,0.004092,0.003023,0.002992,0.002315,0.001974
Mchi-84p0_dMchi-8p0_ctau-10,0.052317,0.011859,0.010159,0.009791,0.009791,0.009791,0.009675,0.006587,0.003410,0.002641,0.002615,0.002136,0.001821
Mchi-73p5_dMchi-7p0_ctau-10,0.045232,0.010180,0.008731,0.008392,0.008392,0.008392,0.008266,0.005492,0.003073,0.002520,0.002483,0.001984,0.001683
Mchi-63p0_dMchi-6p0_ctau-10,0.039166,0.008422,0.007138,0.006830,0.006823,0.006823,0.006745,0.004378,0.002177,0.001862,0.001852,0.001571,0.001337
Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.006813,0.005813,0.005577,0.005575,0.005575,0.005510,0.003545,0.001799,0.001585,0.001573,0.001330,0.001116
Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.005309,0.004514,0.004334,0.004334,0.004334,0.004289,0.002732,0.001396,0.001306,0.001298,0.001143,0.000961
Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.004098,0.003541,0.003413,0.003407,0.003407,0.003372,0.002206,0.001049,0.000962,0.000950,0.000881,0.000751
Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.003087,0.002710,0.002600,0.002597,0.002597,0.002555,0.001585,0.000722,0.000655,0.000650,0.000577,0.000467
Mchi-10p5_dMchi-1p0_ctau-10,0.012434,0.002192,0.001895,0.001780,0.001780,0.001780,0.001732,0.001034,0.000483,0.000432,0.000426,0.000370,0.000317
Mchi-105p0_dMchi-10p0_ctau-10,0.064997,0.015398,0.013211,0.012813,0.012812,0.012812,0.012631,0.008671,0.004976,0.003586,0.003548,0.002620,0.002229


In [23]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '100'
delta = '0p1'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-100,0.041559,0.007310,0.006184,0.006005,0.006004,0.006004,0.005819,0.003501,0.001553,0.001029,0.001022,0.000740,0.000675
Mchi-84p0_dMchi-8p0_ctau-100,0.036959,0.006541,0.005512,0.005337,0.005333,0.005333,0.005176,0.003117,0.001349,0.000917,0.000909,0.000693,0.000614
Mchi-73p5_dMchi-7p0_ctau-100,0.032810,0.005739,0.004859,0.004699,0.004699,0.004699,0.004529,0.002638,0.001051,0.000732,0.000718,0.000555,0.000518
Mchi-63p0_dMchi-6p0_ctau-100,0.027190,0.004479,0.003774,0.003608,0.003608,0.003608,0.003512,0.001943,0.000773,0.000599,0.000596,0.000490,0.000443
Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.003791,0.003235,0.003128,0.003128,0.003128,0.003038,0.001721,0.000625,0.000492,0.000481,0.000397,0.000355
Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.002804,0.002377,0.002301,0.002301,0.002301,0.002233,0.001264,0.000454,0.000368,0.000365,0.000292,0.000262
Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.002086,0.001813,0.001733,0.001733,0.001733,0.001681,0.000920,0.000310,0.000235,0.000229,0.000174,0.000152
Mchi-21p0_dMchi-2p0_ctau-100,0.010649,0.001609,0.001411,0.001343,0.001343,0.001343,0.001289,0.000716,0.000224,0.000166,0.000162,0.000112,0.000095
Mchi-10p5_dMchi-1p0_ctau-100,0.008039,0.001162,0.000986,0.000920,0.000920,0.000920,0.000899,0.000481,0.000162,0.000103,0.000101,0.000068,0.000057
Mchi-105p0_dMchi-10p0_ctau-100,0.046038,0.008477,0.007109,0.006907,0.006906,0.006906,0.006710,0.004013,0.001765,0.001114,0.001098,0.000772,0.000700


In [24]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '1'
delta = '0p2'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-99p0_dMchi-18p0_ctau-1,0.065041,0.019706,0.016500,0.015945,0.015943,0.015943,0.015657,0.011881,0.008091,0.005581,0.005542,0.003345,0.001772
Mchi-77p0_dMchi-14p0_ctau-1,0.049041,0.014407,0.012178,0.011739,0.011739,0.011735,0.011564,0.008743,0.005978,0.004804,0.004753,0.003186,0.001826
Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.002886,0.002380,0.002244,0.002240,0.002240,0.002196,0.001336,0.000717,0.000700,0.000680,0.000643,0.000398
Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.009426,0.007917,0.007607,0.007607,0.007607,0.007518,0.005606,0.003516,0.003294,0.003294,0.002804,0.001559
Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.006941,0.005849,0.005645,0.005642,0.005642,0.005586,0.003974,0.002428,0.002263,0.002230,0.001829,0.001043
Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.005298,0.004643,0.004471,0.004463,0.004463,0.004404,0.003095,0.001919,0.001814,0.001792,0.001595,0.000897
Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.004024,0.003437,0.003258,0.003258,0.003258,0.003220,0.002267,0.001332,0.001287,0.001268,0.001161,0.000706
Mchi-11p0_dMchi-2p0_ctau-1,0.014729,0.003148,0.002666,0.002542,0.002542,0.002542,0.002513,0.001706,0.000922,0.000854,0.000841,0.000793,0.000489
Mchi-110p0_dMchi-20p0_ctau-1,0.070618,0.022157,0.018563,0.017868,0.017861,0.017861,0.017527,0.013300,0.009070,0.005655,0.005577,0.003144,0.001770


In [25]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '10'
delta = '0p2'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-99p0_dMchi-18p0_ctau-10,0.060097,0.015688,0.013278,0.012811,0.012805,0.012805,0.012382,0.008912,0.005614,0.003612,0.003587,0.002134,0.001813
Mchi-88p0_dMchi-16p0_ctau-10,0.051928,0.013361,0.011165,0.010826,0.010826,0.010826,0.010529,0.007516,0.004877,0.003530,0.003489,0.002294,0.001983
Mchi-77p0_dMchi-14p0_ctau-10,0.045542,0.011292,0.009486,0.009167,0.009163,0.009163,0.008949,0.006305,0.003639,0.002832,0.002806,0.001990,0.001685
Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001808,0.001501,0.001404,0.001401,0.001401,0.001321,0.000831,0.000362,0.000331,0.000307,0.000262,0.000217
Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.007399,0.006364,0.006037,0.006037,0.006037,0.005916,0.004096,0.002440,0.002190,0.002164,0.001690,0.001388
Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.005450,0.004600,0.004395,0.004395,0.004395,0.004284,0.002910,0.001632,0.001488,0.001477,0.001191,0.000984
Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.003764,0.003227,0.003107,0.003105,0.003105,0.003040,0.001990,0.001125,0.001067,0.001057,0.000945,0.000814
Mchi-22p0_dMchi-4p0_ctau-10,0.015838,0.002999,0.002563,0.002420,0.002420,0.002420,0.002353,0.001498,0.000778,0.000738,0.000729,0.000652,0.000574
Mchi-11p0_dMchi-2p0_ctau-10,0.012593,0.002179,0.001784,0.001691,0.001691,0.001691,0.001607,0.001005,0.000467,0.000422,0.000417,0.000368,0.000288
Mchi-110p0_dMchi-20p0_ctau-10,0.066991,0.017684,0.014880,0.014421,0.014417,0.014417,0.013928,0.010295,0.006400,0.003872,0.003820,0.002155,0.001852


In [26]:
pd.options.display.float_format = '{:,.6f}'.format

ct = '100'
delta = '0p2'
eff_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-99p0_dMchi-18p0_ctau-100,0.042898,0.007976,0.006672,0.006445,0.006440,0.006440,0.006085,0.003778,0.001806,0.001066,0.001051,0.000583,0.000533
Mchi-88p0_dMchi-16p0_ctau-100,0.038143,0.006857,0.005763,0.005522,0.005515,0.005515,0.005184,0.003321,0.001543,0.000989,0.000978,0.000561,0.000514
Mchi-77p0_dMchi-14p0_ctau-100,0.032554,0.005831,0.004895,0.004672,0.004668,0.004668,0.004420,0.002614,0.001105,0.000771,0.000755,0.000491,0.000454
Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.001127,0.000981,0.000933,0.000933,0.000933,0.000892,0.000466,0.000136,0.000099,0.000089,0.000058,0.000048
Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.003937,0.003324,0.003218,0.003218,0.003218,0.003093,0.001823,0.000725,0.000565,0.000558,0.000388,0.000359
Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.002962,0.002467,0.002359,0.002359,0.002359,0.002252,0.001291,0.000520,0.000432,0.000419,0.000319,0.000286
Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.002093,0.001752,0.001703,0.001703,0.001703,0.001621,0.000962,0.000349,0.000295,0.000290,0.000234,0.000213
Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.001509,0.001295,0.001247,0.001247,0.001247,0.001161,0.000679,0.000262,0.000209,0.000205,0.000154,0.000136
Mchi-11p0_dMchi-2p0_ctau-100,0.007977,0.001088,0.000909,0.000857,0.000857,0.000857,0.000814,0.000445,0.000167,0.000122,0.000113,0.000075,0.000067
Mchi-110p0_dMchi-20p0_ctau-100,0.048055,0.009478,0.007936,0.007632,0.007628,0.007628,0.007249,0.004472,0.002238,0.001235,0.001220,0.000642,0.000593


### Purity = n(Events where sel_vtx is truth matched) / n(Events)

In [29]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '1'
delta = '0p1'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-1,1.0000,0.4250,0.4278,0.4276,0.4276,0.4276,0.4288,0.6006,0.8681,0.9050,0.9161,0.9679,0.9665
Mchi-84p0_dMchi-8p0_ctau-1,1.0000,0.4082,0.4146,0.4177,0.4177,0.4177,0.4184,0.5852,0.8709,0.9047,0.9095,0.9590,0.9591
Mchi-73p5_dMchi-7p0_ctau-1,1.0000,0.3951,0.3991,0.4012,0.4012,0.4012,0.4018,0.5801,0.8624,0.8918,0.9010,0.9693,0.9652
Mchi-63p0_dMchi-6p0_ctau-1,1.0000,0.3854,0.3825,0.3807,0.3807,0.3807,0.3842,0.5537,0.8234,0.8718,0.8840,0.9584,0.9588
Mchi-52p5_dMchi-5p0_ctau-1,1.0000,0.3664,0.3733,0.3760,0.3760,0.3760,0.3772,0.5520,0.8544,0.8925,0.9040,0.9571,0.9578
Mchi-42p0_dMchi-4p0_ctau-1,1.0000,0.3510,0.3565,0.3566,0.3565,0.3565,0.3584,0.5276,0.8329,0.8799,0.8876,0.9567,0.9479
Mchi-31p5_dMchi-3p0_ctau-1,1.0000,0.3267,0.3252,0.3260,0.3260,0.3260,0.3270,0.4834,0.8123,0.8591,0.8620,0.9615,0.9603
Mchi-21p0_dMchi-2p0_ctau-1,1.0000,0.3366,0.3366,0.3404,0.3399,0.3399,0.3414,0.4974,0.8455,0.8884,0.8920,0.9755,0.9663
Mchi-10p5_dMchi-1p0_ctau-1,1.0000,0.3050,0.3080,0.3089,0.3092,0.3092,0.3105,0.4583,0.7971,0.8477,0.8564,0.9516,0.9378
Mchi-105p0_dMchi-10p0_ctau-1,1.0000,0.4398,0.4436,0.4439,0.4441,0.4441,0.4453,0.6156,0.8784,0.9067,0.9137,0.9656,0.9639


In [30]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '10'
delta = '0p1'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-10,1.0000,0.3572,0.3635,0.3650,0.3650,0.3650,0.3665,0.5386,0.8587,0.8975,0.9067,0.9585,0.9614
Mchi-84p0_dMchi-8p0_ctau-10,1.0000,0.3421,0.3413,0.3423,0.3423,0.3423,0.3437,0.4993,0.8549,0.8904,0.8994,0.9532,0.9653
Mchi-73p5_dMchi-7p0_ctau-10,1.0000,0.3356,0.3404,0.3452,0.3452,0.3452,0.3479,0.5221,0.8325,0.8904,0.9039,0.9726,0.9801
Mchi-63p0_dMchi-6p0_ctau-10,1.0000,0.3052,0.3098,0.3094,0.3092,0.3092,0.3107,0.4741,0.8398,0.8818,0.8866,0.9548,0.9620
Mchi-52p5_dMchi-5p0_ctau-10,1.0000,0.3040,0.2996,0.3019,0.3020,0.3020,0.3034,0.4702,0.8369,0.8938,0.9006,0.9657,0.9763
Mchi-42p0_dMchi-4p0_ctau-10,1.0000,0.2874,0.2954,0.3002,0.3002,0.3002,0.3010,0.4688,0.8405,0.8831,0.8882,0.9737,0.9766
Mchi-31p5_dMchi-3p0_ctau-10,1.0000,0.2807,0.2849,0.2881,0.2881,0.2881,0.2893,0.4387,0.8225,0.8931,0.9041,0.9670,0.9742
Mchi-21p0_dMchi-2p0_ctau-10,1.0000,0.2307,0.2287,0.2259,0.2261,0.2261,0.2289,0.3659,0.7371,0.8129,0.8191,0.9220,0.9464
Mchi-10p5_dMchi-1p0_ctau-10,1.0000,0.2270,0.2256,0.2315,0.2315,0.2315,0.2352,0.3760,0.7171,0.8021,0.8139,0.9370,0.9412
Mchi-105p0_dMchi-10p0_ctau-10,1.0000,0.3695,0.3735,0.3734,0.3734,0.3734,0.3748,0.5411,0.8512,0.8841,0.8936,0.9478,0.9569


In [31]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '100'
delta = '0p1'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-100,1.0000,0.1989,0.2079,0.2092,0.2092,0.2092,0.2145,0.3403,0.7033,0.8139,0.8190,0.9364,0.9544
Mchi-84p0_dMchi-8p0_ctau-100,1.0000,0.2043,0.2056,0.2068,0.2062,0.2062,0.2109,0.3316,0.7016,0.7829,0.7901,0.8982,0.9121
Mchi-73p5_dMchi-7p0_ctau-100,1.0000,0.1680,0.1723,0.1743,0.1743,0.1743,0.1800,0.2927,0.6518,0.7624,0.7778,0.8889,0.9231
Mchi-63p0_dMchi-6p0_ctau-100,1.0000,0.1642,0.1686,0.1717,0.1717,0.1717,0.1764,0.2995,0.6785,0.7728,0.7772,0.8959,0.9000
Mchi-52p5_dMchi-5p0_ctau-100,1.0000,0.1492,0.1529,0.1530,0.1530,0.1530,0.1561,0.2604,0.6335,0.7612,0.7773,0.8942,0.9184
Mchi-42p0_dMchi-4p0_ctau-100,1.0000,0.1374,0.1411,0.1415,0.1415,0.1415,0.1448,0.2463,0.6187,0.7574,0.7642,0.9291,0.9585
Mchi-31p5_dMchi-3p0_ctau-100,1.0000,0.1074,0.1064,0.1070,0.1070,0.1070,0.1103,0.1953,0.4893,0.6440,0.6625,0.8487,0.8828
Mchi-21p0_dMchi-2p0_ctau-100,1.0000,0.0808,0.0802,0.0833,0.0833,0.0833,0.0867,0.1525,0.4187,0.5628,0.5763,0.8372,0.9316
Mchi-10p5_dMchi-1p0_ctau-100,1.0000,0.0688,0.0686,0.0685,0.0685,0.0685,0.0701,0.1284,0.3454,0.5410,0.5515,0.8271,0.8636
Mchi-105p0_dMchi-10p0_ctau-100,1.0000,0.2139,0.2231,0.2225,0.2225,0.2225,0.2275,0.3605,0.7303,0.7950,0.8071,0.9168,0.9450


In [32]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '1'
delta = '0p2'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-99p0_dMchi-18p0_ctau-1,1.0000,0.5086,0.5250,0.5257,0.5258,0.5258,0.5320,0.6943,0.8999,0.9199,0.9264,0.9485,0.9433
Mchi-77p0_dMchi-14p0_ctau-1,1.0000,0.4968,0.5065,0.5097,0.5097,0.5099,0.5130,0.6753,0.8894,0.9115,0.9212,0.9652,0.9569
Mchi-5p5_dMchi-1p0_ctau-1,1.0000,0.3193,0.3301,0.3319,0.3324,0.3324,0.3375,0.5447,0.8674,0.8884,0.9151,0.9683,0.9744
Mchi-55p0_dMchi-10p0_ctau-1,1.0000,0.4437,0.4609,0.4681,0.4681,0.4681,0.4736,0.6351,0.8861,0.9189,0.9189,0.9523,0.9427
Mchi-44p0_dMchi-8p0_ctau-1,1.0000,0.4138,0.4263,0.4266,0.4263,0.4263,0.4288,0.6010,0.8795,0.8983,0.9116,0.9622,0.9590
Mchi-33p0_dMchi-6p0_ctau-1,1.0000,0.4224,0.4224,0.4220,0.4221,0.4221,0.4244,0.5990,0.8637,0.8970,0.9082,0.9672,0.9805
Mchi-22p0_dMchi-4p0_ctau-1,1.0000,0.3898,0.4001,0.3972,0.3972,0.3972,0.3989,0.5615,0.8586,0.8851,0.8984,0.9651,0.9595
Mchi-11p0_dMchi-2p0_ctau-1,1.0000,0.3300,0.3416,0.3414,0.3414,0.3414,0.3454,0.5073,0.8338,0.8997,0.9142,0.9688,0.9586
Mchi-110p0_dMchi-20p0_ctau-1,1.0000,0.5011,0.5241,0.5258,0.5259,0.5259,0.5310,0.6957,0.8917,0.9163,0.9283,0.9531,0.9430


In [33]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '10'
delta = '0p1'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-10,1.0000,0.3572,0.3635,0.3650,0.3650,0.3650,0.3665,0.5386,0.8587,0.8975,0.9067,0.9585,0.9614
Mchi-84p0_dMchi-8p0_ctau-10,1.0000,0.3421,0.3413,0.3423,0.3423,0.3423,0.3437,0.4993,0.8549,0.8904,0.8994,0.9532,0.9653
Mchi-73p5_dMchi-7p0_ctau-10,1.0000,0.3356,0.3404,0.3452,0.3452,0.3452,0.3479,0.5221,0.8325,0.8904,0.9039,0.9726,0.9801
Mchi-63p0_dMchi-6p0_ctau-10,1.0000,0.3052,0.3098,0.3094,0.3092,0.3092,0.3107,0.4741,0.8398,0.8818,0.8866,0.9548,0.9620
Mchi-52p5_dMchi-5p0_ctau-10,1.0000,0.3040,0.2996,0.3019,0.3020,0.3020,0.3034,0.4702,0.8369,0.8938,0.9006,0.9657,0.9763
Mchi-42p0_dMchi-4p0_ctau-10,1.0000,0.2874,0.2954,0.3002,0.3002,0.3002,0.3010,0.4688,0.8405,0.8831,0.8882,0.9737,0.9766
Mchi-31p5_dMchi-3p0_ctau-10,1.0000,0.2807,0.2849,0.2881,0.2881,0.2881,0.2893,0.4387,0.8225,0.8931,0.9041,0.9670,0.9742
Mchi-21p0_dMchi-2p0_ctau-10,1.0000,0.2307,0.2287,0.2259,0.2261,0.2261,0.2289,0.3659,0.7371,0.8129,0.8191,0.9220,0.9464
Mchi-10p5_dMchi-1p0_ctau-10,1.0000,0.2270,0.2256,0.2315,0.2315,0.2315,0.2352,0.3760,0.7171,0.8021,0.8139,0.9370,0.9412
Mchi-105p0_dMchi-10p0_ctau-10,1.0000,0.3695,0.3735,0.3734,0.3734,0.3734,0.3748,0.5411,0.8512,0.8841,0.8936,0.9478,0.9569


In [34]:
pd.options.display.float_format = '{:,.4f}'.format

ct = '100'
delta = '0p1'
purities_df[ct][delta].transpose()

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",OSSF,Vertex chi2/df < 3,"dPhi(MET,vtx) < 1",m(ee) < 20,dR(ee) < 1.0,minDxy > 0.01
Mchi-94p5_dMchi-9p0_ctau-100,1.0000,0.1989,0.2079,0.2092,0.2092,0.2092,0.2145,0.3403,0.7033,0.8139,0.8190,0.9364,0.9544
Mchi-84p0_dMchi-8p0_ctau-100,1.0000,0.2043,0.2056,0.2068,0.2062,0.2062,0.2109,0.3316,0.7016,0.7829,0.7901,0.8982,0.9121
Mchi-73p5_dMchi-7p0_ctau-100,1.0000,0.1680,0.1723,0.1743,0.1743,0.1743,0.1800,0.2927,0.6518,0.7624,0.7778,0.8889,0.9231
Mchi-63p0_dMchi-6p0_ctau-100,1.0000,0.1642,0.1686,0.1717,0.1717,0.1717,0.1764,0.2995,0.6785,0.7728,0.7772,0.8959,0.9000
Mchi-52p5_dMchi-5p0_ctau-100,1.0000,0.1492,0.1529,0.1530,0.1530,0.1530,0.1561,0.2604,0.6335,0.7612,0.7773,0.8942,0.9184
Mchi-42p0_dMchi-4p0_ctau-100,1.0000,0.1374,0.1411,0.1415,0.1415,0.1415,0.1448,0.2463,0.6187,0.7574,0.7642,0.9291,0.9585
Mchi-31p5_dMchi-3p0_ctau-100,1.0000,0.1074,0.1064,0.1070,0.1070,0.1070,0.1103,0.1953,0.4893,0.6440,0.6625,0.8487,0.8828
Mchi-21p0_dMchi-2p0_ctau-100,1.0000,0.0808,0.0802,0.0833,0.0833,0.0833,0.0867,0.1525,0.4187,0.5628,0.5763,0.8372,0.9316
Mchi-10p5_dMchi-1p0_ctau-100,1.0000,0.0688,0.0686,0.0685,0.0685,0.0685,0.0701,0.1284,0.3454,0.5410,0.5515,0.8271,0.8636
Mchi-105p0_dMchi-10p0_ctau-100,1.0000,0.2139,0.2231,0.2225,0.2225,0.2225,0.2275,0.3605,0.7303,0.7950,0.8071,0.9168,0.9450
